In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for plotting beautiful graphs
import numbers
import random


# train test split from sklearn
from sklearn.model_selection import train_test_split

# Import Torch 
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F


from PIL import Image, ImageOps, ImageEnhance

In [2]:
# import data
train_df = pd.read_csv('train.csv', dtype=np.float32)
test_df = pd.read_csv('test.csv', dtype=np.float32)

In [3]:
class MNISTDataSet(Dataset):
     def __init__(self, file_path, 
                 transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), 
                     transforms.Normalize(mean=(0.5,), std=(0.5,))])
                ):
        
        df = pd.read_csv(file_path)
        
        if len(df.columns) == n_pixels:
            # test data
            self.X = df.values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        else:
            # training data
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:,0].values)
            
        self.transform = transform
    
     def __len__(self):
        return len(self.X)

     def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])

In [4]:
class RandomRotation(object):
    """
    https://github.com/pytorch/vision/tree/master/torchvision/transforms
    Rotate the image by angle.
    Args:
        degrees (sequence or float or int): Range of degrees to select from.
            If degrees is a number instead of sequence like (min, max), the range of degrees
            will be (-degrees, +degrees).
        resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
        expand (bool, optional): Optional expansion flag.
            If true, expands the output to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
        center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
    """

    def __init__(self, degrees, resample=False, expand=False, center=None):
        if isinstance(degrees, numbers.Number):
            if degrees < 0:
                raise ValueError("If degrees is a single number, it must be positive.")
            self.degrees = (-degrees, degrees)
        else:
            if len(degrees) != 2:
                raise ValueError("If degrees is a sequence, it must be of len 2.")
            self.degrees = degrees

        self.resample = resample
        self.expand = expand
        self.center = center

    @staticmethod
    def get_params(degrees):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        angle = np.random.uniform(degrees[0], degrees[1])

        return angle

    def __call__(self, img):
        """
            img (PIL Image): Image to be rotated.
        Returns:
            PIL Image: Rotated image.
        """
        
        def rotate(img, angle, resample=False, expand=False, center=None):
            """Rotate the image by angle and then (optionally) translate it by (n_columns, n_rows)
            Args:
            img (PIL Image): PIL Image to be rotated.
            angle ({float, int}): In degrees degrees counter clockwise order.
            resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
            expand (bool, optional): Optional expansion flag.
            If true, expands the output image to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
            center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
            """
                
            return img.rotate(angle, resample, expand, center)

        angle = self.get_params(self.degrees)

        return rotate(img, angle, self.resample, self.expand, self.center)

In [5]:
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)

In [6]:
batch_size = 64
n_test = len(test_df)
n_pixels = len(test_df.columns)

# data sets
train_dataset = MNISTDataSet('train.csv', transform= transforms.Compose(
                            [transforms.ToPILImage(),
                             RandomRotation(degrees=20), 
                             RandomShift(3),
                             transforms.ToTensor(), 
                             transforms.Normalize(mean=(0.5,), std=(0.5,))
                            ]))
test_dataset = MNISTDataSet('test.csv')

# data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.features = nn.Sequential(
            # Convolution 1
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            
            # Convolution 1
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25),
            
            # Convolution 3
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),

            # Convolution 4
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            
            # Convolution 5
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            
            # Convolution 6
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(1152, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(512,10)
        )
                                     
        
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1) # flatten
        out = self.classifier(out)
        return out

print(CNNModel())

CNNModel(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.25, inplace=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): Conv2d(128, 128, kernel_size=(3, 3), stride=(

In [8]:
# set up optimizer, loss function, lr scheduler
model = CNNModel()
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()
expr_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [9]:
def train(epoch):
    model.train()
    expr_lr_scheduler.step()
    
    for i, (features, labels) in enumerate(train_loader):
        features, labels = Variable(features), Variable(labels)
        if torch.cuda.is_available():
            features = features.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        # forward propagation
        output = model(features)
        
        # calculate loss
        loss = criterion(output, labels)

        # backprop
        loss.backward()
        
        # update params (gradient descent)
        optimizer.step()
        
        if (i + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (i + 1) * len(features), len(train_loader.dataset),
                100. * (i + 1) / len(train_loader), loss.data.item()))

In [10]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for features, labels in data_loader:
        with torch.no_grad():
            features, labels = Variable(features), Variable(labels)
            if torch.cuda.is_available():
                features = features.cuda()
                labels = features.cuda()
            output = model(features)
            
        loss += F.cross_entropy(output, labels, size_average=False).data.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
    
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [11]:
n_epochs = 12

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

/Users/dvm/Library/Python/3.7/lib/python/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [6400/42000 (15%)]	Loss: 0.400175
Train Epoch: 0 [12800/42000 (30%)]	Loss: 0.469981
Train Epoch: 0 [19200/42000 (46%)]	Loss: 0.356844
Train Epoch: 0 [25600/42000 (61%)]	Loss: 0.208510
Train Epoch: 0 [32000/42000 (76%)]	Loss: 0.060391
Train Epoch: 0 [38400/42000 (91%)]	Loss: 0.184861


/Users/dvm/Library/Python/3.7/lib/python/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Average loss: 0.2077, Accuracy: 39410/42000 (93.833%)

Train Epoch: 1 [6400/42000 (15%)]	Loss: 0.202743
Train Epoch: 1 [12800/42000 (30%)]	Loss: 0.097602
Train Epoch: 1 [19200/42000 (46%)]	Loss: 0.172268
Train Epoch: 1 [25600/42000 (61%)]	Loss: 0.156035
Train Epoch: 1 [32000/42000 (76%)]	Loss: 0.280107
Train Epoch: 1 [38400/42000 (91%)]	Loss: 0.089095

Average loss: 0.1037, Accuracy: 40655/42000 (96.798%)

Train Epoch: 2 [6400/42000 (15%)]	Loss: 0.198771
Train Epoch: 2 [12800/42000 (30%)]	Loss: 0.129867
Train Epoch: 2 [19200/42000 (46%)]	Loss: 0.113441
Train Epoch: 2 [25600/42000 (61%)]	Loss: 0.169901
Train Epoch: 2 [32000/42000 (76%)]	Loss: 0.029279
Train Epoch: 2 [38400/42000 (91%)]	Loss: 0.147399

Average loss: 0.0827, Accuracy: 40917/42000 (97.421%)

Train Epoch: 3 [6400/42000 (15%)]	Loss: 0.104483
Train Epoch: 3 [12800/42000 (30%)]	Loss: 0.069888
Train Epoch: 3 [19200/42000 (46%)]	Loss: 0.208507
Train Epoch: 3 [25600/42000 (61%)]	Loss: 0.008002
Train Epoch: 3 [32000/42000 (76%)]	

In [12]:
def prediction(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        with torch.no_grad():
            data = Variable(data)
            if torch.cuda.is_available():
                data = data.cuda()
            output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [13]:
test_pred = prediction(test_loader)
out_df = pd.DataFrame(np.c_[np.arange(1, len(test_dataset)+1)[:,None], test_pred.numpy()], 
                      columns=['ImageId', 'Label'])
out_df.to_csv('submission2.csv', index=False)